In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training_3.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
test = pd.concat([test,postulaciones])
test = test.drop_duplicates(['idaviso','idpostulante'])
postulaciones = test[100000:].drop('id', axis=1)
test = test[:100000].drop('id', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1112248724,NjlD,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
1,1112411485,NjlD,0.0,Part-time,Senior / Semi-Senior,Logística,Brinks Argentina S.A.,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
2,1112349301,NjlD,0.0,Full-time,Senior / Semi-Senior,Análisis Funcional,Humana Partners,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
3,1112393228,NjlD,0.0,Full-time,Senior / Semi-Senior,Selección,Jones Lang LaSalle,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
4,1112326525,NjlD,0.0,Full-time,Senior / Semi-Senior,Gerencia / Dirección General,Adecco -Región GBA SUR,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM


In [5]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,739260,6M9ZQR,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1976-03-29,FEM
1,739260,6v1xdL,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1987-06-27,MASC
2,739260,ezRKm9,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1982-02-17,FEM
3,758580,1Q35ej,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1949-12-24,MASC
4,758580,EAN4J6,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1986-05-27,FEM


In [6]:
def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        #date_new = int(date_new)
        return date_new
    except ValueError:
        return -1

In [7]:
fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [8]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    print(col)
    postulaciones[col] = postulaciones[col].fillna('na')
    test[col] = test[col].fillna('na')
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1,4,120,2440,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1
1,2,4,117,553,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1
2,1,4,6,2030,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1
3,1,4,165,2237,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1
4,1,4,83,211,0.0,0.0,0.0,0.0,0.0,0.0,3.0,37.0,1


In [10]:
postulaciones.sample(50)

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
14823535,1,4,30,1121,0.0,0.0,2.0,0.0,0.0,3.0,3.0,31.0,2
6338726,1,4,185,4210,0.0,0.0,0.0,0.0,0.0,0.0,3.0,22.0,2
15895335,1,1,90,1343,0.0,0.0,0.0,0.0,0.0,0.0,2.0,23.0,2
4715976,1,4,30,922,0.0,0.0,3.0,0.0,3.0,0.0,0.0,26.0,2
7428910,7,2,168,1018,0.0,0.0,0.0,0.0,0.0,0.0,2.0,21.0,1
3300851,1,4,30,1620,0.0,0.0,0.0,0.0,3.0,3.0,0.0,26.0,1
2518220,2,4,174,3251,0.0,0.0,0.0,0.0,3.0,0.0,2.0,20.0,2
5808827,1,4,92,3700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,1
9997355,1,4,117,2905,0.0,0.0,0.0,0.0,3.0,0.0,0.0,41.0,2
4749510,2,2,42,1636,0.0,0.0,0.0,0.0,3.0,0.0,2.0,49.0,2


In [11]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [12]:
aviso = None
postulantes = None
postulaciones.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=100,n_jobs=-1)
clf.fit(postulaciones,resultado)
y_pred=clf.predict(test)



In [14]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = y_pred
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

In [16]:
y_pred[y_pred==1]

array([1., 1., 1., ..., 1., 1., 1.])